In [35]:
import imageio
import numpy as np
import gym
from gym.utils.play import play

pygame 2.1.2 (SDL 2.0.18, Python 3.9.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


/usr/local/lib/python3.9/site-packages/gym/utils/play.py:11: UserWarning: WARN: failed to set matplotlib backend, plotting will not work: No module named '_tkinter'
  logger.warn("failed to set matplotlib backend, plotting will not work: %s" % str(e))


After analyzing the image game we found :
    - image 210 x 160 x 3
    - ball 4 x 2
    - pad 4 * 16

We have decided to consider the game like a pong prototype, our goal is to send back the ball.
As we decided to use Q-learning. Our training are made from a discrete environment.
We first have to reduce our environment :
Only keep bottom from the paddle and before the first breakable block (see image -> ....)

In [36]:
def callback(obs_t, obs_tp1, action, rew, done, info):
    # image  60 x 72 x 1 color.
    imageio.imwrite('outfile1.png', obs_t[94:194:2, 8:152:2, 1])


env = gym.make('Breakout-v0')
play(env, zoom=3, fps=12, callback=callback)
env.reset()
env.close()

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


After reduction,we have:
   - image ->  60 x 72 x 1
   - ball 2 x 1
   - pad 2 * 8

Our Q state is composed of :
       B - ball position
       P - pad position
       A - action possible : 3  ( left , stay , right )
       D - direction
N  = BxQxNxD
The final goal will be to try with different composition of B P D . In the aim to find which discretization is more efficiant

Before generalization. Let's make a first example.
We discretize our image :
60 and 72 can be divided both by 12.
For the direction of the ball we take 4 directions, bottom left - bottom right - top left - top right
Our Q state is composed of :
       B - ball position :  5 x 6 position = 30
       P - pad position : 6 position only horizontal
       A - action possible : 3  ( left , stay , right )
       D - direction  : 4
N = 30 x 6 x 3 x 4 = 2160

In [85]:
import numpy as np
import random

XB = 6
YB = 5
P = 6
A = 3
D = 4  #  considering : 0 top-left  1 top right ...
Q = np.zeros((P, XB, YB, D, A), dtype=float)
alpha = 0.8
# x,y,direction
moveBall = {
    0: (1, 1),  # 45 top right
    1: (1, -1),  # -45 bottom right
    2: (-1, -1),  # -135 bottom left
    3: (-1, 1)  # 135 top-left
}


def updateBall(x, y, d, pad):
    #
    xD, yD = moveBall[d]
    nd = d
    nX, nY = x + xD, y + yD

    if pad == nX and nY == 0 or YB - 1 == nY:
        nd = (d + 2) % 4

    # if
    elif (XB - 1 == nX and d == 0) or (nX == 0 and d == 3):
        nd = d + (-3 if XB - 1 == nX and d == 0 else +3)


    elif (XB - 1 == nX and d == 1) or (nX == 0 and d == 2):
        nd = d + (1 if XB - 1 == nX and d == 1 else -1)



    return nX, nY, nd


def rewards(xb, yb, p):
    if p == xb and yb == 0:
        return 3
    return -5 if newYB == 0 and newDB in [1, 2] else 0


...

# iterations

N = 1000
for i in range(N):
    ## Create the pad here random position =
    xPad = 2
    epsilon = 1
    xBall, yBall, dirBall = 2, 2, 3
    endStep = 12000
    while not ((yBall == 0 and dirBall in [1, 2] ) or endStep ==0 )  :

        # find the index of the max of the line of Q_state
        maxIndex =  np.where(Q[xPad][xBall][yBall][dirBall] == max(Q[xPad][xBall][yBall][dirBall]))[0]
        #    np.argmax(Q[xPad][xBall][yBall][dirBall])

        # e greedy
        r = random.random()
        action = random.choice([0, 1, 2] if r < epsilon else maxIndex)
        epsilon -= 0.05

        newXP = xPad + (action - 1 if P - 1 > xPad > 0 else 0)
        newXB, newYB, newDB = updateBall(xBall, yBall, dirBall, newXP)
        reward = rewards(newXB, newYB, newXP)

        Q[xPad][xBall][yBall][dirBall][action] += alpha * (
                    reward + max(Q[newXP][newXB][newYB][newDB]) - Q[xPad][xBall][yBall][dirBall][action])

        xPad = newXP
        xBall, yBall, dirBall = newXB, newYB, newDB
        endStep-=1
     #   print("position pad :", xPad, "position ball :", xBall, yBall, dirBall)

print(Q)




[[[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

   [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

   [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

   [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

   [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [-5.00000000e+00 -5.00000000e+00 -5.00000000e+00]
    [ 0.00000000e+00

In [38]:
def Qlearning(B, P, A, D, ballPos):
    ...


def direction(direction):
    #return {i:360/direction}
    ...
